### Import Libraries

In [43]:
import os
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from datetime import datetime
import numpy as np

### Get All FR file path

In [2]:
fr_excel_OP_folder = ""

In [3]:
SCV_Global_folder = ""

In [4]:
def find_fr_excel_files(root_dir, search_criteria):
    fr_files_dict = {}
    
    # Traverse the directory tree
    for dirpath, dirnames, filenames in os.walk(root_dir):
        for filename in filenames:
            # Check if the file ends with 'FR.xlsx'
            if filename.endswith(search_criteria):
                # Add the filename as the key and the directory as the value
                file_path = os.path.join(dirpath, filename)
                fr_files_dict[filename] = file_path
    
    return fr_files_dict

In [5]:
def create_point_geometry(row):
    return Point((row['Longitude'], row['Latitude']))

def remove_fr_extension(filename, substring):
    # Remove the 'FR.xlsx' substring from the filename
    return filename.replace(substring, "")

# Function to clean spaces in the columns
def clean_spaces(df, columns):
    df[columns] = df[columns].apply(lambda x: x.str.strip().str.replace(r'\s+', ' ', regex=True))
    return df

def combine_excel_files(fr_files_dict):
    combined_gdf = pd.DataFrame()
    number_of_files_processed = 0
    total_fr_files = 0
    unprocessed_filename = []

    for filename, dirpath in fr_files_dict.items():
        file_path = dirpath
        # Print the filename (key in dictionary)
        print(f"Processing file: {filename}")
        total_fr_files += 1
        
        try:
            # Read the 'FARMERS', 'PLOTS', and 'ANALYSIS' sheets
            farmers_df = pd.read_excel(file_path, sheet_name="FARMERS")
            plots_df = pd.read_excel(file_path, sheet_name="PLOTS")
            analysis_df = pd.read_excel(file_path, sheet_name="ANALYSIS")

            # Perform a left join between 'FARMERS' and 'PLOTS' on 'Farmer ID'
            farmer_plot_combined_df = pd.merge(plots_df, farmers_df, on="Farmer ID", how="left")

            # Perform a left join between the resulting dataframe and 'ANALYSIS' on 'Farmer ID'
            PFA_combined_df = pd.merge(farmer_plot_combined_df,
                                        analysis_df,left_on = "Plot ID",right_on="Plot ID", how="left")

            if 'Longitude' in PFA_combined_df.columns and 'Latitude' in PFA_combined_df.columns:
                # Create the 'Point' column by applying the 'create_point' function to each row
                # PFA_combined_df['Point'] = PFA_combined_df.apply(create_point_geometry, axis=1)
                filename_stripped_xlsx = remove_fr_extension(filename , "FR.xlsx")
                PFA_combined_df['Group'] = filename_stripped_xlsx
                PFA_combined_df = clean_spaces(PFA_combined_df, ["Group"])
                # gdf = gpd.GeoDataFrame(PFA_combined_df, geometry='Point', crs="EPSG:4326")
                gdf = pd.DataFrame(PFA_combined_df)


                # Append the GeoDataFrame to the combined GeoDataFrame
                combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)
                number_of_files_processed += 1
            else:
                print(f"Missing 'Longitude' or 'Latitude' columns in 'PLOTS' sheet of file {filename}")
        except Exception as e:
            unprocessed_filename.append(filename)
            print(f"Error processing {filename}: {e}")
    # combined_gdf.set_geometry("Point",inplace =True)
    print(f"Processed {number_of_files_processed}/{total_fr_files} Farmers Registered Files Successful")
    print(f"Error while processing these files {unprocessed_filename}")
    return combined_gdf

### WORK FLOW

In [6]:
# Set the root directory where the search will begin
root_directory = fr_excel_OP_folder 
search_extension = "FR.xlsx"
# Get all FR.xlsx files in a dictionary
fr_files_dict = find_fr_excel_files(root_directory , search_extension)
fr_files_dict = dict(list(fr_files_dict.items())[:])
fr_files_dict

{'01-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\01-COL-2024\\01-COL-2024 FR.xlsx',
 '02-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\02-COL-2024\\02-COL-2024 FR.xlsx',
 '03-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\03-COL-2024\\03-COL-2024 FR.xlsx',
 '04-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\04-COL-2024\\04-COL-2024 FR.xlsx',
 '05-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\05-COL-2024\\05-COL-2024 FR.xlsx',
 '06-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\06-COL-2024\\06-COL-2024 FR.xlsx',
 '07-COL-2024 FR.xlsx': 'C:\\Users\\XPS\\SUCRES ET DENREES\\TM-Sustainability - 02_Country\\01_Colombia\\07-COL-2024\\07-COL-2024 FR.xlsx',
 '08-COL-2024 FR.xls

In [7]:
combined_fr_df = combine_excel_files(fr_files_dict)

Processing file: 01-COL-2024 FR.xlsx
Processing file: 02-COL-2024 FR.xlsx
Processing file: 03-COL-2024 FR.xlsx
Processing file: 04-COL-2024 FR.xlsx
Processing file: 05-COL-2024 FR.xlsx
Processing file: 06-COL-2024 FR.xlsx
Processing file: 07-COL-2024 FR.xlsx
Processing file: 08-COL-2024 FR.xlsx
Processing file: 09-COL-2024 FR.xlsx
Processing file: 10-COL-2024 FR.xlsx
Processing file: 11-COL-2024 FR.xlsx
Processing file: 12-COL-2024 FR.xlsx
Processing file: 13-COL-2024 FR.xlsx
Processing file: 14-COL-2024 FR.xlsx
Processing file: 15-COL-2024 FR.xlsx
Processing file: 16-COL-2024 FR.xlsx
Processing file: 17-COL-2024 FR.xlsx
Processing file: 18-COL-2024 FR.xlsx
Processing file: 19-COL-2024 FR.xlsx
Processing file: 20-COL-2024 FR.xlsx
Processing file: 23-COL-2024 FR.xlsx
Processing file: 24-COL-2024 FR.xlsx
Processing file: 25-COL-2024 FR.xlsx
Processing file: 26-COL-2024 FR.xlsx
Processing file: 27-COL-2024 FR.xlsx
Processing file: 28-COL-2024 FR.xlsx
Processing file: 29-COL-2024 FR.xlsx
P

C:\Users\XPS\AppData\Local\Temp\ipykernel_21616\1107944987.py:49: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_gdf = pd.concat([combined_gdf, gdf], ignore_index=True)


Processing file: 01-IDN-2024 FR.xlsx
Processing file: 04-IDN-2024 FR.xlsx
Processing file: 01-VNM-2024 FR.xlsx
Processing file: 02-VNM-2024 FR.xlsx
Processing file: 03-VNM-2024 FR.xlsx
Processing file: 04-VNM-2024 FR.xlsx
Processed 39/39 Farmers Registered Files Successful
Error while processing these files []


In [8]:
combined_fr_df.describe()

,Area,Latitude,Longitude,Number of Plot of Land for this farm,Total farm area (ha),Certified crop area (ha),% mapped,Plot to be visited,Area_x,percentage,[ROBUSTA] Total harvest of previous year (kg),[ROBUSTA] Volume sold/delivered to the management in previous year (kg)
count,10514.000000,10514.000000,10514.000000,10514.000000,10514.000000,10514.000000,10514.0,10514.000000,10514.000000,10514.000000,7427.000000,7427.000000
mean,2.482936,4.291683,51.567383,1.528248,4.942257,3.325660,1.0,1.469279,2.482936,0.649505,5569.058604,3515.550122
std,28.916491,7.789596,82.018027,0.940750,55.932078,20.340269,0.0,0.723732,28.916491,3.966033,16659.990553,15776.263740
min,0.020000,-21.582369,-76.235311,1.000000,0.166667,0.130000,1.0,1.000000,0.020000,0.000000,0.000000,0.000000
25%,0.666667,-4.072337,-75.630669,1.000000,1.000000,0.890000,1.0,1.000000,0.666667,0.000000,900.000000,0.000000
50%,1.000000,2.452101,103.250266,1.000000,1.500000,1.225000,1.0,1.000000,1.000000,0.000000,1500.000000,0.000000
75%,1.500000,13.765216,107.828169,2.000000,3.000000,2.260000,1.0,2.000000,1.500000,0.000000,5580.000000,3994.000000
max,2511.560000,14.080180,108.138372,10.000000,3564.000000,894.370000,1.0,4.000000,2511.560000,100.000000,794397.800000,794397.800000


### Save to SCV Staging Folder

In [15]:
def get_today_date_time():
    # Get the current date and time
    now = datetime.now()
    # Format it to the desired format
    formatted_date_time = now.strftime("%y%m%d_%H%M%S")
    return formatted_date_time

In [16]:
SCV_sub_folders = ["SCV_Archive", "SCV_Dataset", "SCV_Staging"]
# Get current timestamp for backup file naming
timestamp = get_today_date_time()

# File paths
PFA_staging_csv = os.path.join(SCV_sub_folders[2], "PFA_Staging_Data.csv")
PFA_main_dataset_csv = os.path.join(SCV_sub_folders[1], "PFA_Combined_Data.csv")
PFA_dataset_backup_csv = os.path.join(SCV_sub_folders[0], f"{timestamp}_PFA_data.csv")

# Full directories
PFA_staging_directory = os.path.join(SCV_Global_folder, PFA_staging_csv)
PFA_main_dataset_directory = os.path.join(SCV_Global_folder, PFA_main_dataset_csv)
PFA_dataset_backup_directory = os.path.join(SCV_Global_folder, PFA_dataset_backup_csv)

# Output the directories
print(PFA_staging_directory, "\n", PFA_main_dataset_directory, "\n", PFA_dataset_backup_directory)

C:\Users\XPS\OneDrive - SUCRES ET DENREES\Power BI\SV_GLOBAL_Dashboard\SCV_Staging\PFA_Staging_Data.csv 
 C:\Users\XPS\OneDrive - SUCRES ET DENREES\Power BI\SV_GLOBAL_Dashboard\SCV_Dataset\PFA_Combined_Data.csv 
 C:\Users\XPS\OneDrive - SUCRES ET DENREES\Power BI\SV_GLOBAL_Dashboard\SCV_Archive\241218_093722_PFA_data.csv


Save Staging

In [17]:
PFA_staging_df = combined_fr_df.copy()
PFA_staging_df.to_csv(PFA_staging_directory , index = False)

Back Up

In [18]:
PFA_main_df = pd.read_csv(PFA_main_dataset_directory)
PFA_main_df.to_csv(PFA_dataset_backup_directory, index = False)

Staging to Final

In [52]:
def extract_and_map_country(input_string):
    # Dictionary mapping
    mapping_dict = {
        "COL": "Columbia",
        "BRA": "Brazil",
        "VNM": "Vietnam",
        "IDN": "Indonesia",
        "IND": "India"
    }
    extracted_value = input_string[3:6] 
    return mapping_dict.get(extracted_value, "Unknown")

# Function to process the 'Gender' column
def process_gender(df):
    df = df[df["Gender"].notna()]  # Remove rows where Gender is NaN
    df['Gender'] = df['Gender'].str.replace(' ', '', regex=False)
    df['Gender'] = df['Gender'].replace({'woman': 'Female', 'man': 'Male'}, regex=True)
    return df

# Function to apply title case to specific columns
def apply_title_case(df, columns):
    df[columns] = df[columns].apply(lambda x: x.str.title())
    return df

# Function to map values based on a dictionary (assuming 'extract_and_map_country' is already defined)
def map_country(df, column, mapping_function):
    df['Country'] = df[column].apply(mapping_function)
    return df
def new_sample_column(df):
    df['SampleColumn1'] = 1
    df['SampleColumn2'] = 1
    df['SampleColumn3'] = 1
    return df

# Main processing function
def process_dataset(df):
    PFA_columns_order = ['Farmer ID', 'Plot ID', 'Area', 'Latitude', 'Longitude', 'Polygon',
       'Deforestation Risk', 'Valid Farmer ID', 'Valid Plot ID',
       'Geolocation Required', 'EUDR compatible', 'Def_Status', 'Name',
       'Surname', 'Phone number', 'Gender', 'Birth Date', 'Village / City',
       'District/State/Province', 'Region', 'Farm type',
       'Number of Plot of Land for this farm', 'Total farm area (ha)',
       'Certified crop area (ha)', 'Species', 'Processing', 'Mill',
       '[ARABICA] Total harvest estimation of current year (kg)',
       '[ARABICA] Total harvest of previous year (kg)',
       '[ARABICA] Volume sold/delivered to the management in previous year (kg)',
       'Unique ID', '% mapped', 'Selected for Audit', 'Plot to be visited', 'percentage', 'Group',
       '[ROBUSTA] Total harvest estimation of current year (kg)',
       '[ROBUSTA] Total harvest of previous year (kg)',
       '[ROBUSTA] Volume sold/delivered to the management in previous year (kg)']
    
    # Step 1: Reorder columns
    df = df[PFA_columns_order]
    
    # Step 2: Process Gender column
    df = process_gender(df)
    
    # Step 3: Apply title case to specific columns
    title_edit_columns = ['Village / City', 'District/State/Province', 'Region']
    df = apply_title_case(df, title_edit_columns)
    
    # Step 4: Clean extra spaces in multiple columns
    columns_to_clean_spaces = ['Name', 'Surname', 'Gender', 'Village / City', 'District/State/Province', 'Region']
    df = clean_spaces(df, columns_to_clean_spaces)
    
    # Step 5: Get Country name
    df = map_country(df, 'Group', extract_and_map_country) 

    df = new_sample_column(df)
    
    # Fill na to 0
    volume_columns_to_fillna = ['[ARABICA] Total harvest estimation of current year (kg)',
       '[ARABICA] Total harvest of previous year (kg)',
       '[ARABICA] Volume sold/delivered to the management in previous year (kg)',
       '[ROBUSTA] Total harvest estimation of current year (kg)',
       '[ROBUSTA] Total harvest of previous year (kg)',
       '[ROBUSTA] Volume sold/delivered to the management in previous year (kg)']
    df[volume_columns_to_fillna] = df[volume_columns_to_fillna].replace(r'^\s*-+\s*$', np.nan, regex=True)
    df[volume_columns_to_fillna] = df[volume_columns_to_fillna].fillna(0)
    
    return df

In [53]:
PFA_staging_csv_df = pd.read_csv(PFA_staging_directory)
PFA_final_dataset = PFA_staging_csv_df.copy()
PFA_final_dataset = process_dataset(PFA_final_dataset)
# PFA_final_dataset.to_csv("a.csv",index = False)
PFA_final_dataset.to_csv(PFA_main_dataset_directory,index = False)

In [ ]:
PFA_final_dataset.head(10)

,Farmer ID,Plot ID,Area,Latitude,Longitude,Polygon,Deforestation Risk,Valid Farmer ID,Valid Plot ID,Geolocation Required,...,Plot to be visited,percentage,Group,[ROBUSTA] Total harvest estimation of current year (kg),[ROBUSTA] Total harvest of previous year (kg),[ROBUSTA] Volume sold/delivered to the management in previous year (kg),Country,SampleColumn1,SampleColumn2,SampleColumn3
0,1666465,1666465-F01,2.55,1.973480,-75.966000,Yes,Low,Yes,Yes,Point,...,2.0,1.8,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
1,1666465,1666465-F02,1.52,1.972811,-75.965731,Yes,Low,Yes,Yes,Point,...,2.0,4.6,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
2,2483370,2483370-F01,0.65,1.939060,-75.916440,Yes,Low,Yes,Yes,Point,...,2.0,0.4,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
3,2483370,2483370-F02,0.54,1.940689,-75.920450,Yes,Low,Yes,Yes,Point,...,2.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
4,4900622,4900622-F01,1.35,1.975360,-76.000120,NaN,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
5,4929350,4929350-F01,0.58,2.071250,-75.958440,NaN,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
6,4938092,4938092-F01,1.62,1.945870,-75.960010,Yes,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
7,4946940,4946940-F01,1.35,1.950610,-75.898800,Yes,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
8,4947548,4947548-F01,0.71,1.948500,-75.960820,Yes,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
9,4947651,4947651-F01,3.13,1.984440,-75.986070,NaN,Low,Yes,Yes,Point,...,1.0,0.0,01-COL-2024,0,0.0,0.0,Columbia,1,1,1
